# Simple ReAct Agent from Scratch

In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

C:\Users\Ali Computers\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# Configure Gemini with your API key from .env
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Initialize the Gemini model (use 1.5 if available in your account)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Send a prompt to Gemini
response = model.generate_content("What is AI? define in 1 line ")

# Print the response
print(response.text)

AI is the simulation of human intelligence processes by machines, especially computer systems.



In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append(f"[System instruction]: {system}")

        # Load the Gemini model (you can also use 'gemini-pro' or other)
        self.model = genai.GenerativeModel(model_name="gemini-1.5-flash")

    def __call__(self, message):
        self.messages.append(f"[User]: {message}")
        result = self.execute()
        self.messages.append(f"[Assistant]: {result}")
        return result

    def execute(self):
        prompt = "\n".join(self.messages)
        response = self.model.generate_content(prompt)
        return response.text

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [6]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [7]:
abot = Agent(prompt)

In [8]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I need to find the average weight of a toy poodle.  I can use the `average_dog_weight` action for this.

Action: average_dog_weight: Toy Poodle
PAUSE



In [9]:
result = average_dog_weight("Toy Poodle")

In [10]:
result

'a toy poodles average weight is 7 lbs'

In [11]:
next_prompt = "Observation: {}".format(result)

In [12]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs\n'

In [13]:
abot.messages

['[System instruction]: You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs',
 '[User]: How much does a toy poodle weigh?',
 '[Assistan

In [14]:
abot = Agent(prompt)

In [15]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add those weights together.  I'll use the `average_dog_weight` action for each breed.\n\nAction: average_dog_weight: Border Collie\nPAUSE\n"

In [16]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [17]:
abot(next_prompt)

'Thought: I have the average weight of a Border Collie. Now I need to find the average weight of a Scottish Terrier and add the two weights together.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE\n'

In [18]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [19]:
abot(next_prompt)

'[Assistant]: Thought: I have the average weight for both breeds. Now I just need to sum them to get the combined weight.\n\nAction: calculate: 37 + 20\n\nPAUSE\n'

In [20]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [21]:
abot(next_prompt)

'Answer: The combined weight of the two dogs is 57 lbs.\n'